In [1]:
"""
Purpose: to classify a neuron as inhibitory or excitatory
based on the following rules:


Have 3 Groups: 1) Sparsely spined, 2) Densely spined 3) No spi
For the sparsely spined group: 
1) Look for axon (Excitatory)
2) Look for long apical up (Excitatory)

--> then can also error out some axon like parts

"""

'\nPurpose: to classify a neuron as inhibitory or excitatory\nbased on the following rules:\n\n\nHave 3 Groups: 1) Sparsely spined, 2) Densely spined 3) No spi\nFor the sparsely spined group: \n1) Look for axon (Excitatory)\n2) Look for long apical up (Excitatory)\n\n--> then can also error out some axon like parts\n\n'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from os import sys
sys.path.append("/meshAfterParty/")

In [4]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

import proofreading_utils as pru

import preprocessing_vp2 as pre

# -- For the axon classification --

import neuron_searching as ns

import skeleton_utils as sk

import numpy_utils as nu

import networkx_utils as xu

import system_utils as su

INFO - 2021-01-24 02:06:34,564 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-24 02:06:34,565 - settings - Setting database.user to celiib
INFO - 2021-01-24 02:06:34,566 - settings - Setting database.password to newceliipass
INFO - 2021-01-24 02:06:34,577 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-24 02:06:34,578 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-24 02:06:34,591 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-24 02:06:34,861 - settings - Setting enable_python_native_blobs to True


In [5]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-24 02:06:34,954 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-24 02:06:35,242 - settings - Setting enable_python_native_blobs to True


# Importing Neuron

In [6]:
minnie.Decomposition()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134219060901,0,0.25,30.00,0,0,1,=BLOB=,11723,24809,0,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.8309
864691134242673681,0,0.25,30.00,0,0,1,=BLOB=,10252,20204,0,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.9755
864691134252908410,0,0.25,30.00,0,0,1,=BLOB=,10910,22764,0,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.2558
864691134316523453,0,0.25,30.00,0,0,1,=BLOB=,10031,22403,0,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.5114
864691134384033872,0,0.25,30.00,0,0,1,=BLOB=,10015,20509,0,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.8075
864691134391593458,0,0.25,30.00,0,0,1,=BLOB=,11947,25033,0,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.7341
864691134409056935,0,0.25,30.00,0,0,1,=BLOB=,10770,22630,0,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.9198
864691134428609977,0,0.25,30.00,0,0,1,=BLOB=,13062,28555,0,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.3236
864691134441691943,0,0.25,30.00,0,0,1,=BLOB=,10681,23432,0,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.9167
864691134484023255,0,0.25,30.00,0,0,1,=BLOB=,21063,46992,0,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.2306


In [7]:
segment_id = 864691135269732773 # didn't work because mistakenly found axon (which was at weird angle)

In [8]:
neuron_obj = (minnie.Decomposition() & dict(segment_id=segment_id)).fetch1("decomposition")
#neuron_obj.calculate_spines()

Decompressing Neuron in minimal output mode...please wait
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-7daeda2690f0>", line 1, in <module>
    neuron_obj = (minnie.Decomposition() & dict(segment_id=segment_id)).fetch1("decomposition")
  File "/usr/local/lib/python3.6/dist-packages/datajoint/fetch.py", line 245, in __call__
    result = self._expression.proj(*attributes).fetch(squeeze=squeeze, download_path=download_path)
  File "/usr/local/lib/python3.6/dist-packages/datajoint/fetch.py", line 203, in __call__
    ret[name] = list(map(partial(get, heading[name]), ret[name]))
  File "/usr/local/lib/python3.6/dist-packages/datajoint/fetch.py", line 54, in _get
    return adapt(extern.download_filepath(uuid.UUID(bytes=data))[0])
  File "/meshAfterParty/datajoint_utils.py", line 266, in get
    original_mesh=dec_mesh)
 

TypeError: object of type 'NoneType' has no len()

In [33]:
curr_file = "./864691135269732773_sparse_inhibitory"
neuron_obj = nru.decompress_neuron(curr_file,
                                  curr_file)

hiiiii new method
Decompressing Neuron in minimal output mode...please wait


In [34]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [67]:
# neuron_obj.save_compressed_neuron(file_name=f"{neuron_obj.segment_id}_sparse_inhibitory",
#                                  export_mesh=True)

Saving Neuorn in suppress_output mode...please wait
Saved File at location: 864691135657796994_sparse_inhibitory


In [10]:
nviz.plot_spines(neuron_obj,
                mesh_whole_neuron_alpha=0.3,
                mesh_spines_alpha=1)


 Working on visualization type: mesh

 Working on visualization type: skeleton


# Part 1: Sorting into Categories :How to sort them into different categories

In [30]:
import classification_utils as clu
clu = reload(clu)

In [31]:
ret_value = clu.inhibitory_excitatory_classifier(neuron_obj,return_axon_angles=True,
                                                return_spine_classification=True,
                                                axon_angle_threshold_relaxed = 80,
                        axon_angle_threshold = 90,
                                               return_axon_apical_flags=True )

spine_category = sparsely_spined
Did not pass final filters to continuing
NO APICAL BRANCHES TO PLOT

Working on Limb L0

Working on Limb L3

Working on Limb L4

 Working on visualization type: mesh


apical_flag = False
axon_flag = True
axon_angles = {3: {0: 95.72679030495159}}


In [32]:
ret_value

['excitatory', 'sparsely_spined', {3: {0: 95.72679030495159}}, True, False]

In [19]:
axon_labels,axon_angles,error_labels = clu.axon_classification(neuron_obj,verbose=True,
                       plot_candidates=True,
                       plot_skeleton_paths_after_restriction=False,
                        axon_angle_threshold_relaxed = 80,
                        axon_angle_threshold = 90,
                       return_axon_angles=True)


Part 1: Axon like branchese 
{'L0': array([ 2,  3, 12, 23, 27]), 'L1': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 13, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 43, 44, 49, 50, 51, 52, 53, 54, 55, 56, 57,
       58, 59, 60, 61]), 'L2': array([0, 1, 2, 3, 5, 9]), 'L3': array([0, 1, 2, 3, 4]), 'L4': array([0])}
Limb L0 soma angle: 123.88 
Limb L1 soma angle: 38.74 
Limb L2 soma angle: 53.14 
Limb L3 soma angle: 80.39 
Limb L4 soma angle: 72.41 

Part 2: possible_axon_limbs = [0, 3, 4]

Working on Limb L0
nodes_to_eliminate = [19]

Working on Limb L3
nodes_to_eliminate = []

Working on Limb L4
nodes_to_eliminate = []

----Working on Limb L0-----
No endpoints were eliminated so don't need to add back any candidates

----Working on Limb L3-----
No nodes were eliminated so don't need to add back any candidates

----Working on Limb L4-----
No nodes were eliminated so don't need to add back any candidat

Final Candidates

Limb 0
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17])
array([12, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28])

Limb 3
array([0, 1, 2, 3, 4])

Limb 4
array([0])
Part 3: axon_subgraph_candidates = {0: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17], [12, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28]], 3: [[0, 1, 2, 3, 4]], 4: [[0]]}


 --- Working on limb 0, candidate # 0
2 out of 17 branches are axons
Axon percentage = 0.11764705882352941
curr_axon_angle_threshold = 90
endnodes_to_test = [5268 6145 6808 7190 7279]
Skeleton paths distances = [24951.9165756  24976.55895749 24957.37402246 24882.20356812
 24907.13218611]
Filtered indexes = [0 1 2 3 4]
len(filtered_skeletons) = 5
sk_angles = [127.87494595 150.87881896 150.87881896 155.72687836 109.03655145]
Not adding candidate the first branch was not an axon 


 --- Working on limb 0, candidate # 1
3 out of 11 branches are axons
Axon percentage = 0.2727272727272727
curr_axon_angle_threshold 